In [ ]:
! pip install -r requirements.txt

In [1]:
# Manipulation de données
import pandas as pd
import numpy as np

# Traitement du signal
from scipy import signal
import mne

# Machine Learning et Deep Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import torch


# Visualisation
import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px

# Gestion de Notebooks
#import papermill as pm
import ipywidgets as widgets

# Utilitaires
import joblib
import yaml
import pickle
import os
import sys
import importlib

# Importation code local
sys.path.append('preprocessing')
import preprocess
import dataset
#import torchcam
#import models.GGN.ggn_model as GGN
#import models.GGN.train as train
#importlib.reload(preprocess)
#importlib.reload(GGN)
#importlib.reload(train)
#importlib.reload(dataset)

print('hello')

hello


## Chargement de la configuration du projet

In [3]:
# Chargement de la configuration YAML
with open("config.yml", "r") as file:
    config = yaml.safe_load(file)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import os
print(f"Répertoire courant : {os.getcwd()}")

Répertoire courant : /project/166600089/mathis-add-noises


## Noisy adjustements

In [26]:
import os
import mne
import numpy as np

def add_noise_to_data(data, noise_level=0.1):
    """
    Ajoute un bruit gaussien aux données EEG.
    Arguments :
    - data : ndarray, les données EEG (n_epochs, n_channels, n_times).
    - noise_level : float, intensité du bruit (fraction de l'amplitude max des données).
    Retourne :
    - data_noisy : ndarray, les données avec du bruit ajouté.
    """
    noise = np.random.normal(0, noise_level * np.max(data), size=data.shape)
    return data + noise




In [25]:
def process_and_save_noisy_data(base_dir, output_dir, noise_level=0.1):
    """
    Parcourt les fichiers .fif, ajoute du bruit et sauvegarde les fichiers bruités dans le sous-dossier parent.
    Arguments :
    - base_dir : str, répertoire contenant les fichiers originaux.
    - output_dir : str, répertoire où sauvegarder les fichiers bruités.
    - noise_level : float, intensité du bruit.
    """
    if not os.path.exists(base_dir):
        raise FileNotFoundError(f"Le chemin spécifié n'existe pas : {base_dir}")
    
    print(f"Chemin de base : {base_dir}")
    print(f"Contenu du répertoire : {os.listdir(base_dir)}")
    
    fif_files = [
        os.path.join(root, file)
        for root, dirs, files in os.walk(base_dir)
        for file in files if file.endswith("-epo.fif")
    ]
    
    if not fif_files:
        print("Aucun fichier .fif trouvé dans le répertoire spécifié.")
        return
    
    print(f"Nombre de fichiers .fif trouvés : {len(fif_files)}")
    
    for root, dirs, files in os.walk(base_dir):
        print(f"Exploration du répertoire : {root}")
        for file in files:
            if file.endswith("-epo.fif"):
                print(f"Traitement du fichier : {file}")
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, base_dir)
                noisy_path = os.path.join(output_dir, relative_path)  # Dossier parent sans "noisy"
                os.makedirs(noisy_path, exist_ok=True)

                try:
                    # Charger les données
                    epochs = mne.read_epochs(file_path, preload=True)
                    data = epochs.get_data()
                    print(f"Shape des données : {data.shape}")
                    
                    # Ajouter du bruit
                    data_noisy = add_noise_to_data(data, noise_level)
                    epochs._data = data_noisy
                    
                    # Sauvegarder les données bruitées dans le dossier parent
                    noisy_file_path = os.path.join(noisy_path, file)
                    epochs.save(noisy_file_path, overwrite=True)
                    print(f"Fichier bruité sauvegardé : {noisy_file_path}")
                except Exception as e:
                    print(f"Erreur lors du traitement du fichier {file}: {e}")


In [ ]:
# Configuration
base_dir = os.path.join(os.getcwd(), "data/processed")  # Répertoire de base contenant les fichiers originaux
output_dir = os.path.join(os.getcwd(), "data/noisy") # Répertoire de sortie pour les fichiers bruités

print(base_dir)
print(output_dir)
noise_level = 0.3  # Intensité du bruit

process_and_save_noisy_data(base_dir, output_dir, noise_level)

In [7]:
# Appel de la fonction split

from preprocessing.preprocess import train_test_split_files
path_noisy = "data/noisy"
path_split = "data/split_noisy"

train_test_split_files(path_noisy, path_split, 0.2, 42)

Processing subject: sub-022
Processing task: audiopassive
Saved splits for sub-022 task audiopassive in data/split_noisy
Processing task: audioactive
Saved splits for sub-022 task audioactive in data/split_noisy
Processing task: thermalpassive
Saved splits for sub-022 task thermalpassive in data/split_noisy
Processing task: thermalactive
Saved splits for sub-022 task thermalactive in data/split_noisy
Processing subject: sub-028
Processing task: thermalpassive
Saved splits for sub-028 task thermalpassive in data/split_noisy
Processing task: audiopassive
Saved splits for sub-028 task audiopassive in data/split_noisy
Processing task: audioactive
Saved splits for sub-028 task audioactive in data/split_noisy
Processing task: thermalactive
Saved splits for sub-028 task thermalactive in data/split_noisy
Processing subject: sub-010
Processing task: audiopassive
Saved splits for sub-010 task audiopassive in data/split_noisy
Processing task: audioactive
Saved splits for sub-010 task audioactive 

PermissionError: [Errno 1] Operation not permitted: 'data/split_noisy/train/sub-003/audioactive/16-epo.fif'

## Model Training

In [ ]:
running_model = config['project_config']['running_model']

subjects_id = config['data']['subjects']

bids_root = config['data']['path']


test_losses = []
accuracies = []
recalls = []
precisions = []
f1_scores = []
auc_rocs = []

if not subjects_id:
        subjects_id = [
            d for d in os.listdir(bids_root)
            if os.path.isdir(os.path.join(bids_root, d)) and d.startswith("sub-")
        ]
        print(f"Aucun ID de sujet spécifié. Tous les sujets détectés : {subjects_id}")

for subject in subjects_id:

    if running_model == "GGN":
        train_loader, val_loader, test_loader = dataset.create_dataloader([subject], config)

        model = GGN.GGN(**config['models']['GGN']['parameters'], device=device)
        
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 50

        # Train and validate the model
        train.train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

        # Test the model
        avg_test_loss, accuracy, recall, precision, f1, auc_roc = train.test(model, test_loader, criterion, device)
        test_losses.append(avg_test_loss)
        accuracies.append(accuracy)
        recalls.append(recall)
        precisions.append(precision)
        f1_scores.append(f1)
        auc_rocs.append(auc_roc)
        
        model.explain_temporal_cnn(test_loader, device)
        

    elif running_model == "SVM":
        train_loader, val_loader, test_loader = dataset.create_dataloader([subject], config)

        # Convert data loaders to numpy arrays
        X_train, y_train = preprocess.dataloader_without_topology_to_numpy(train_loader)
        X_val, y_val = preprocess.dataloader_without_topology_to_numpy(val_loader)
        X_test, y_test = preprocess.dataloader_without_topology_to_numpy(test_loader)
        
        print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
        print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
        print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

        # Reshape data
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_val = X_val.reshape(X_val.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        # Train the SVM
        svm_params = config['models']['SVM']['parameters']
        svm = SVC(**svm_params)
        svm.fit(X_train, y_train)

        # Evaluate on validation set
        y_val_pred = svm.predict(X_val)
        val_acc = accuracy_score(y_val, y_val_pred)

        # Test the model
        y_test_pred = svm.predict(X_test)
        test_acc = accuracy_score(y_test, y_test_pred)
        accuracies.append(test_acc)

# Calculate mean test accuracy across all subjects
mean_test_loss = np.mean(test_losses)
mean_accuracy = np.mean(accuracies)
mean_recall = np.mean(recalls)
mean_precision = np.mean(precisions)
mean_f1 = np.mean(f1_scores)
mean_auc_roc = np.mean(auc_rocs)

print(f"Mean Test Loss across all subjects: {mean_test_loss:.4f}")
print(f"Mean Test Accuracy across all subjects: {mean_accuracy:.2f}%")
print(f"Mean Recall (Sensitivity) across all subjects: {mean_recall:.2f}")
print(f"Mean Precision across all subjects: {mean_precision:.2f}")
print(f"Mean F1 Score across all subjects: {mean_f1:.2f}")
print(f"Mean AUC-ROC across all subjects: {mean_auc_roc:.2f}")
